In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

device: cuda:0


In [3]:
from google.colab import files

# Make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Next, install the Kaggle API client.
!pip install -q kaggle

# The Kaggle API client expects the json file to be in ~/.kaggle,
# so move it there.
!rm -rf ~/.kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json


-rw-r--r-- 1 root root 69 Feb 11 08:29 kaggle.json


In [4]:
!kaggle datasets download -d gpiosenka/sports-classification

# You might want to unzip the dataset if it's compressed.
!unzip sports-classification.zip >> /dev/null


 99% 421M/424M [00:17<00:00, 22.9MB/s]
100% 424M/424M [00:17<00:00, 24.9MB/s]


In [5]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to the desired size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

In [6]:
# train data loader and test data loader
dataset = ImageFolder(root='./train', transform=transform)

batch_size = 128
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageFolder(root='./test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [28]:
def train_and_return_test_accuracy(model, num_epochs=30, learning_rate=0.005):

    sgd_optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    model = model.to(device)

    # Iterative training process
    for epoch in range(num_epochs):
        for batch_index, (inputs, labels) in enumerate(train_loader):

            inputs, labels = inputs.to(device), labels.to(device)

            sgd_optimizer.zero_grad()
            predictions = model(inputs)
            loss_value = criterion(predictions, labels)
            loss_value.backward()
            sgd_optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Training Epoch: {epoch + 1} \tLoss: {loss_value.item():.6f}')

    # Evaluating accuracy on the test dataset
    correct_predictions = 0
    all_samples = 0

    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs, labels = inputs.to(device), labels.to(device)

            test_outputs = model(inputs)
            _, preds = torch.max(test_outputs.data, 1)
            all_samples += labels.size(0)
            correct_predictions += (preds == labels).sum().item()

    accuracy_on_test = correct_predictions / all_samples
    return accuracy_on_test

In [29]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
# modify the last layer to fit the number of classes 100
model.classifier[6] = nn.Linear(4096, 100)
model = model.to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [30]:
test_accuracy = train_and_return_test_accuracy(model)

Training Epoch: 10 	Loss: 3.972022
Training Epoch: 20 	Loss: 2.374878
Training Epoch: 30 	Loss: 1.255746
Test accuracy: 0.506 % 


In [33]:
print(f'Test accuracy: {100 * test_accuracy} % ')

Test accuracy: 50.6 % 


### 2. Transfer learning


In [34]:
def train_and_return_test_accuracy(model, num_epochs=5, learning_rate=0.005):

    sgd_optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    model = model.to(device)

    # Iterative training process
    for epoch in range(num_epochs):
        for batch_index, (inputs, labels) in enumerate(train_loader):

            inputs, labels = inputs.to(device), labels.to(device)

            sgd_optimizer.zero_grad()
            predictions = model(inputs)
            loss_value = criterion(predictions, labels)
            loss_value.backward()
            sgd_optimizer.step()

        if (epoch + 1) % 1 == 0:
            print(f'Training Epoch: {epoch + 1} \tLoss: {loss_value.item():.6f}')

    # Evaluating accuracy on the test dataset
    correct_predictions = 0
    all_samples = 0

    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs, labels = inputs.to(device), labels.to(device)

            test_outputs = model(inputs)
            _, preds = torch.max(test_outputs.data, 1)
            all_samples += labels.size(0)
            correct_predictions += (preds == labels).sum().item()

    accuracy_on_test = correct_predictions / all_samples
    return accuracy_on_test


In [35]:
# Transfer learning
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
# modify the last layer to fit the number of classes 100
model.classifier[6] = nn.Linear(4096, 100)

# freeze all feature layers
for param in model.features.parameters():
    param.requires_grad = False

model = model.to(device)

test_accuracy = train_and_return_test_accuracy(model)
print(f'Test accuracy: {100 * test_accuracy} % ')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 146MB/s]


Training Epoch: 1 	Loss: 0.622307
Training Epoch: 2 	Loss: 0.625878
Training Epoch: 3 	Loss: 0.279085
Training Epoch: 4 	Loss: 0.628623
Training Epoch: 5 	Loss: 0.341436
Test accuracy: 81.39999999999999 % 


### 3. Transfer learning based on vgg19


In [36]:
def train_and_return_test_accuracy(model, num_epochs=5, learning_rate=0.005):

    sgd_optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    model = model.to(device)

    # Iterative training process
    for epoch in range(num_epochs):
        for batch_index, (inputs, labels) in enumerate(train_loader):

            inputs, labels = inputs.to(device), labels.to(device)

            sgd_optimizer.zero_grad()
            predictions = model(inputs)
            loss_value = criterion(predictions, labels)
            loss_value.backward()
            sgd_optimizer.step()

        if (epoch + 1) % 1 == 0:
            print(f'Training Epoch: {epoch + 1} \tLoss: {loss_value.item():.6f}')

    # Evaluating accuracy on the test dataset
    correct_predictions = 0
    all_samples = 0

    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs, labels = inputs.to(device), labels.to(device)

            test_outputs = model(inputs)
            _, preds = torch.max(test_outputs.data, 1)
            all_samples += labels.size(0)
            correct_predictions += (preds == labels).sum().item()

    accuracy_on_test = correct_predictions / all_samples
    return accuracy_on_test

In [37]:
vgg19 = models.vgg19(pretrained=True)
vgg19.classifier[6] = nn.Linear(4096, 100)

# freeze all feature layers
for param in vgg19.features.parameters():
    param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 186MB/s]


In [38]:
vgg19 = vgg19.to(device)

test_accuracy = train_and_return_test_accuracy(vgg19)
print(f'Test accuracy: {100 * test_accuracy} % ')

Training Epoch: 1 	Loss: 0.851092
Training Epoch: 2 	Loss: 0.521460
Training Epoch: 3 	Loss: 0.484028
Training Epoch: 4 	Loss: 0.308592
Training Epoch: 5 	Loss: 0.299722
Test accuracy: 91.8 % 
